# Big Data Cup Event #3

In [73]:
import pandas as pd
import numpy as np
import re
import nltk
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.2.


In [3]:
import os
os.getcwd()

"c:\\Users\\mcinn\\OneDrive - Queen's University\\GMMA 865 - Big Data Analytics\\865BigData"

# Load Data

In [6]:
df = pd.read_csv("spamraw_train.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   sms_text  5000 non-null   object
 2   spam      5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


,id,sms_text,spam
0,1,Hope you are having a good week. Just checking in,0
1,2,K..give back my thanks.,0
2,3,Am also doing in cbe only. But have to pay.,0
3,4,"complimentary 4 STAR Ibiza Holiday or £10,000 ...",1
4,5,okmail: Dear Dave this is your final notice to...,1


In [7]:
np.bincount(df['spam'])

array([4327,  673], dtype=int64)

In [1]:
a = 0 
for i in range(a,a+4):
    print(df['sms_text'][i])
    print()

def clean_text_round1(text):
    '''Make text lowercase, remove text in brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\[^\w\s]','', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1_train = lambda x: clean_text_round1(x)
df_clean_train = pd.DataFrame(df.sms_text.apply(round1_train))
df_clean_train.head()


NameError: name &#39;df&#39; is not defined

In [166]:
stop_words = set(stopwords.words('english')) 
for i, line in enumerate(df_clean_train.sms_text): 
    df_clean_train.sms_text[i] = ' '.join([x for 
        x in nltk.word_tokenize(line) if 
        ( x not in stop_words )]) 
df_clean_train

,sms_text
0,hope good week . checking
1,k .. give back thanks .
2,also cbe . pay .
3,"complimentary star ibiza holiday £ , cash need..."
4,okmail : dear dave final notice collect * tene...
...,...
4995,aiyar dun disturb u liao ... thk u lots aft ur...
4996,"sms services . inclusive text credits , pls go..."
4997,'re car hour half 'm going crazy
4998,mm umma ask vava also come tell play later tog...


In [167]:
df_clean_train['id'] = df['id']
df_clean_train['spam'] = df['spam']
df_clean_train.head()

,sms_text,id,spam
0,hope good week . checking,1,0
1,k .. give back thanks .,2,0
2,also cbe . pay .,3,0
3,"complimentary star ibiza holiday £ , cash need...",4,1
4,okmail : dear dave final notice collect * tene...,5,1


# Build Pipeline

In [176]:
from sklearn.model_selection import train_test_split

X = df_clean_train['sms_text']
y = df_clean_train['spam']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [169]:
type(X_train)
X_train.shape
X_train.head()

type(y_train)
y_train.shape
y_train.head()

pandas.core.series.Series

(4500,)

3716                           u workin overtime nigpun ?
3779               call times job today ok umma ask speed
135                                       eatin lunch ...
4480    unique user id . removal send stop customer se...
1437                             keep payasam rinu brings
Name: sms_text, dtype: object

pandas.core.series.Series

(4500,)

3716    0
3779    0
135     0
4480    1
1437    0
Name: spam, dtype: int64

In [122]:
# Simple preprocessor.
# Input is a single document, as a single string.
# Otuput should be a single document, as a single string.
#def my_preprocess(doc):
    
    # Lowercase
    #doc = doc.lower()
    
    # Remove URLs
    #doc = re.sub(r'http\S+', '', doc)
        
    # TODO: What else?
    #doc = re.sub('\[.*?\]', '', doc)
    #doc = re.sub('\[^\w\s]','', doc)
    #doc = re.sub('\w*\d\w*', '', doc)
    #return doc

In [129]:
#from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from sklearn.tree import DecisionTreeClassifier

#vectorizer = CountVectorizer(max_features = 15, preprocessor = my_preprocess)
#dt = DecisionTreeClassifier(max_depth=3, random_state=20)

#pipe = Pipeline([('cv', vectorizer), ('clf', dt)])




# Other things to try above:
# - More/Different preprocessing steps
# - Different hyperparameter values for CountVectorizer
# - Different hyperparameter values for DecisionTreeClassifier
# - Different ML algorithms
# - Additional feature engineering (can be added to Pipeline via sklearn.pipeline.FeatureUnion)

In [198]:

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
svm = make_pipeline(CountVectorizer(ngram_range=(1,2)),SGDClassifier(loss='hinge', penalty='l2', alpha=0.0025, random_state=42), ).fit(X_train, y_train)
#pred_val = svm.predict(X_val)



In [132]:
#pipe.fit(X_train, y_train)

Pipeline(steps=[('cv',
                 CountVectorizer(max_features=15,
                                 preprocessor=<function my_preprocess at 0x000001F281D9E168>)),
                ('clf', DecisionTreeClassifier(max_depth=3, random_state=20))])

# Estimate Model Performance

In [199]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score

pred_val = svm.predict(X_val)

print("Confusion matrix:")
print(confusion_matrix(y_val, pred_val))

print("\nF1 Score = {:.5f}".format(f1_score(y_val, pred_val, average="micro")))

print("\nClassification Report:")
print(classification_report(y_val, pred_val))

Confusion matrix:
[[441   0]
 [  7  52]]

F1 Score = 0.98600

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       441
           1       1.00      0.88      0.94        59

    accuracy                           0.99       500
   macro avg       0.99      0.94      0.96       500
weighted avg       0.99      0.99      0.99       500



# Create Submission File on Kaggle Hold-Out Data

In [181]:
test_df = pd.read_csv('spamraw_test.csv')

pred_test = svm.predict(test_df['sms_text'])

# Output the predictions to a file to upload to Kaggle.
my_submission = pd.DataFrame({'id': test_df.id, 'predicted': pred_test})
my_submission.head()
my_submission.to_csv('my_submission_v5.csv', index=False)

,id,predicted
0,12000,1
1,12001,0
2,12002,0
3,12003,0
4,12004,0
